In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from tqdm import tqdm

# ------------------------------
# Utils
# ------------------------------
def to_decimal(x):
    """
    Accept:
    - 0.0054
    - '0.54%'
    - 0.54   -> interpreted as %
    """
    if isinstance(x, str):
        x = x.strip()
        if x.endswith("%"):
            return float(x[:-1]) / 100
        return float(x)
    x = float(x)
    return x / 100 if x > 1 else x

def monthly_close(close: pd.Series) -> pd.Series:
    # use ME to avoid pandas warning
    return close.dropna().resample("ME").last()

def monthly_returns(monthly_close: pd.Series) -> pd.Series:
    return monthly_close.pct_change().dropna()

def compute_beta(ri: pd.Series, rm: pd.Series) -> float:
    """
    β = Cov(Ri, Rm) / Var(Rm)
    """
    df = pd.concat([ri, rm], axis=1).dropna()
    if len(df) < 12:
        return np.nan

    cov = np.cov(df.iloc[:, 0], df.iloc[:, 1], ddof=1)[0, 1]
    var = np.var(df.iloc[:, 1], ddof=1)

    return cov / var if var != 0 else np.nan

def annualize(r_m):
    return (1 + r_m) ** 12 - 1

# ------------------------------
# CAPM (STATIC INPUTS) !!!
# ------------------------------
def capm_static_inputs(
    tickers,
    market_ticker="^GSPC",
    start="2019-01-01",
    end="2023-12-01",
    er_m_monthly="0.66%",   # E(Rm) Monthly (consider Low, Base, High) adjust risk profile
    rf_monthly="0.29%"     # Rf from 3-month T-Bill, STATIC
):
    er_m = to_decimal(er_m_monthly)
    rf = to_decimal(rf_monthly)

    # Market returns (for beta estimation only)
    mkt = yf.download(
        market_ticker,
        start=start,
        end=end,
        auto_adjust=True,
        progress=False
    )
    if mkt.empty:
        raise ValueError("Market data not found")

    rm = monthly_returns(monthly_close(mkt["Close"]))

    results = []

    for t in tqdm(tickers, desc="CAPM", unit="ticker"):
        try:
            px = yf.download(
                t,
                start=start,
                end=end,
                auto_adjust=True,
                progress=False
            )
            if px.empty:
                results.append([t, np.nan, np.nan, np.nan, 0])
                continue

            ri = monthly_returns(monthly_close(px["Close"]))

            df = pd.concat([ri, rm], axis=1).dropna()
            n = len(df)

            if n < 12:
                results.append([t, np.nan, np.nan, np.nan, n])
                continue

            beta = compute_beta(df.iloc[:, 0], df.iloc[:, 1])

            # CAPM (MONTHLY)
            er_i_m = rf + beta * (er_m - rf)

            results.append([
                t,
                beta,
                er_i_m,
                annualize(er_i_m),
                n
            ])

        except Exception:
            results.append([t, np.nan, np.nan, np.nan, 0])

    df_out = pd.DataFrame(
        results,
        columns=[
            "Ticker",
            "Beta",
            "E(Ri)_Monthly",
            "E(Ri)_Annualized",
            "N_Months"
        ]
    )

    return df_out.sort_values("E(Ri)_Annualized", ascending=False).reset_index(drop=True)

# ------------------------------
# Example
# ------------------------------
if __name__ == "__main__":
    tickers = ["NVDA","AAPL","GOOG","GOOGL","MSFT","AMZN","META","TSLA","AVGO","BRK-B",
    "LLY","WMT","JPM","V","ORCL","MA","JNJ","XOM","NFLX","PLTR",
    "BAC","ABBV","COST","HD","PG","AMD","GE","KO","CVX","UNH",
    "CSCO","WFC","IBM","MS","GS","CAT","AXP","MU","PM","MRK",
    "CRM","RTX","MCD","ABT","TMUS","PEP","C","DIS","ISRG","LIN",
    "AMAT","LRCX","BX","INTU","QCOM","AMGN","BLK","TJX","T","VZ",
    "BKNG","INTC","SCHW","ACN","NEE","GEV","UBER","NOW","TXN","DHR",
    "BA","APH","SPGI","ANET","KLAC","COF","GILD","ADBE","PFE","UNP",
    "BSX","LOW","SYK","PGR","ADI","DE","WELL","PANW","HON","MDT",
    "CB","ETN","PLD","CRWD","KKR","COP","VRTX","CMCSA","LMT","MDLZ",
    "NOC","GD","CI","TGT","SNPS","CDNS","EMR","PH","AON","CL",
    "ITW","MET","CSX","DUK","MAR","MMC","FTNT","KMB","WM","ORLY",
    "ROP","BDX","ROST","SO","WDAY","PCAR","TEAM","SLB","SYY","PAYX",
    "TRV","EW","CTSH","OTIS","AEP","GWW","MSCI","DLTR","HPQ","HCA",
    "EOG","VRSK","YUM","NTAP","FAST","COR","KR","AME","ZBRA","PRU",
    "XYL","A","EBAY","SHW","SWKS","EFX","EXC","HSY","STX","IQV",
    "URI","LEN","AIG","FIS","KDP","IR","MPC","GM","ALGN","DFS",
    "JCI","ECL","BBY","HPE","LUV","XEL","CLX","UAL","CFG","NVR",
    "VMC","BIO","CHRW","RCL"]

    df = capm_static_inputs(
        tickers=tickers,
        er_m_monthly="0.66%", # E(Rm) Monthly (Tentukan Low, Base, High) sesuai profil risiko
        rf_monthly="0.29%" # Rf dari T-Bill 3 bulan, STATIC
    )

    # Pretty display (%)
    view = df.copy()
    for c in ["E(Ri)_Monthly", "E(Ri)_Annualized"]:
        view[c] = (view[c] * 100).round(2).astype(str) + "%"

    print(view)


CAPM:  35%|███▌      | 65/184 [00:21<00:39,  3.05ticker/s]
1 Failed download:
['GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2023-12-01) (Yahoo error = "Data doesn\'t exist for startDate = 1546318800, endDate = 1701406800")')
CAPM:  92%|█████████▏| 169/184 [00:55<00:04,  3.49ticker/s]
1 Failed download:
['DFS']: YFTzMissingError('possibly delisted; no timezone found')
CAPM: 100%|██████████| 184/184 [01:00<00:00,  3.06ticker/s]

    Ticker      Beta E(Ri)_Monthly E(Ri)_Annualized  N_Months
0     PLTR  2.763981         1.31%           16.94%        38
1      RCL  2.527942         1.23%           15.74%        58
2     TSLA  2.459387          1.2%           15.39%        58
3      URI  1.816308         0.96%           12.18%        58
4     ZBRA  1.764029         0.94%           11.92%        58
..     ...       ...           ...              ...       ...
179   VRTX  0.336576         0.41%            5.09%        58
180    LLY  0.324666         0.41%            5.03%        58
181   GILD  0.189127         0.36%            4.41%        58
182    GEV       NaN          nan%             nan%         0
183    DFS       NaN          nan%             nan%         0

[184 rows x 5 columns]


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import plotly.graph_objects as go

# ------------------------------
# Utils
# ------------------------------
def to_decimal(x):
    if isinstance(x, str):
        s = x.strip()
        if s.endswith("%"):
            return float(s[:-1]) / 100.0
        return float(s)
    x = float(x)
    return x / 100.0 if x > 1.0 else x

def monthly_close(close: pd.Series) -> pd.Series:
    return close.dropna().resample("ME").last()  # ME avoids pandas warning

def monthly_returns_from_close(close: pd.Series) -> pd.Series:
    return monthly_close(close).pct_change().dropna()

# ------------------------------
# Plot SML (Monthly) from df CAPM
# ------------------------------
def plot_sml_monthly(df_capm: pd.DataFrame, # Input Tanggal awal, akhir, dan E(Rm) dan Rf
                     start="2019-01-01",
                     end="2023-12-01",
                     er_m_monthly="0.66%",
                     rf_monthly="0.29%"):
    """
    SML line uses static inputs:
      SML(beta) = Rf + beta*(E(Rm)-Rf)
    Points use realized mean monthly returns (historical) vs beta from df_capm.
    """
    er_m = to_decimal(er_m_monthly)
    rf = to_decimal(rf_monthly)
    mrp = er_m - rf

    tickers = df_capm["Ticker"].dropna().unique().tolist()

    # Realized mean monthly returns (historical)
    rows = []
    for t in tqdm(tickers, desc="Realized Mean Monthly", unit="ticker"):
        px = yf.download(t, start=start, end=end, auto_adjust=True, progress=False)
        if px is None or px.empty:
            rows.append([t, np.nan])
            continue

        ri = monthly_returns_from_close(px["Close"])
        rows.append([t, ri.mean().item() if not ri.empty else np.nan])  # <- no FutureWarning

    df_real = pd.DataFrame(rows, columns=["Ticker", "Realized_Monthly_Mean"])

    # Merge with CAPM output
    df_plot = (
        df_capm.merge(df_real, on="Ticker", how="left")
              .dropna(subset=["Beta", "Realized_Monthly_Mean"])
              .copy()
    )

    # SML (CAPM) monthly at each beta, and alpha
    df_plot["SML_CAPM_Monthly"] = rf + df_plot["Beta"] * mrp
    df_plot["Alpha_Monthly"] = df_plot["Realized_Monthly_Mean"] - df_plot["SML_CAPM_Monthly"]

    # SML line range
    x_min = min(0.0, float(df_plot["Beta"].min()) - 0.2)
    x_max = float(df_plot["Beta"].max()) + 0.2
    x_line = np.linspace(x_min, x_max, 200)
    y_line = rf + x_line * mrp

    # Color points by alpha sign (green above SML, red below)
    colors = np.where(df_plot["Alpha_Monthly"] >= 0, "green", "red")

    fig = go.Figure()

    # SML line
    fig.add_trace(go.Scatter(
        x=x_line, y=y_line,
        mode="lines",
        name="SML (monthly)",
        hovertemplate="Beta=%{x:.3f}<br>SML=%{y:.4%}<extra></extra>"
    ))

    # Points
    fig.add_trace(go.Scatter(
        x=df_plot["Beta"],
        y=df_plot["Realized_Monthly_Mean"],
        mode="markers+text",
        text=df_plot["Ticker"],
        textposition="top center",
        name="Stocks (realized mean monthly)",
        marker=dict(size=10, color=colors),
        customdata=np.column_stack([
            df_plot["SML_CAPM_Monthly"],
            df_plot["Alpha_Monthly"],
            df_plot.get("N_Months", pd.Series([np.nan]*len(df_plot)))
        ]),
        hovertemplate=(
            "Ticker=%{text}<br>"
            "Beta=%{x:.3f}<br>"
            "Realized=%{y:.4%}<br>"
            "SML(CAPM)=%{customdata[0]:.4%}<br>"
            "Alpha=%{customdata[1]:.4%}<br>"
            "N=%{customdata[2]}<extra></extra>"
        )
    ))

    fig.update_layout(
        title=f"SML (Monthly) — Rf={rf:.2%}, E(Rm)={er_m:.2%}, MRP={mrp:.2%}",
        xaxis_title="Beta (vs market)",
        yaxis_title="Monthly Return",
        template="plotly_white",
        height=650
    )

    # Return a table sorted by alpha (descending) + plot
    df_alpha = df_plot.sort_values("Alpha_Monthly", ascending=False).reset_index(drop=True)
    return df_alpha, fig


# ------------------------------
# (df from CAPM tab)
# ------------------------------
# df = capm_static_inputs(...)

df_alpha, fig = plot_sml_monthly( #Input start date, end date, E(Rm) dan Rf
    df_capm=df,
    start="2019-01-01",
    er_m_monthly="0.66%",
    rf_monthly="0.29%"
)

print(df_alpha[["Ticker","Beta","Realized_Monthly_Mean","SML_CAPM_Monthly","Alpha_Monthly"]])
fig.show()


Realized Mean Monthly:  99%|█████████▉| 182/184 [00:53<00:00,  3.01ticker/s]
1 Failed download:
['GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2023-12-01) (Yahoo error = "Data doesn\'t exist for startDate = 1546318800, endDate = 1701406800")')
Realized Mean Monthly:  99%|█████████▉| 183/184 [00:53<00:00,  3.05ticker/s]
1 Failed download:
['DFS']: YFTzMissingError('possibly delisted; no timezone found')
Realized Mean Monthly: 100%|██████████| 184/184 [00:54<00:00,  3.38ticker/s]


    Ticker      Beta  Realized_Monthly_Mean  SML_CAPM_Monthly  Alpha_Monthly
0     TSLA  2.459387               0.065298          0.012000       0.053298
1     PLTR  2.763981               0.060365          0.013127       0.047239
2     NVDA  1.689045               0.055641          0.009149       0.046492
3      AMD  1.602037               0.038607          0.008828       0.029779
4     CRWD  1.049328               0.035868          0.006783       0.029086
..     ...       ...                    ...               ...            ...
177    DIS  1.398917               0.002159          0.008076      -0.005917
178     BA  1.544729               0.001584          0.008615      -0.007031
179    FIS  0.949052              -0.004256          0.006411      -0.010667
180    UAL  1.531560              -0.003059          0.008567      -0.011626
181    LUV  1.114243              -0.007426          0.007023      -0.014449

[182 rows x 5 columns]
